# 数据归档

## 配置信息

In [3]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import os
pymysql.install_as_MySQLdb()

# 修改当前主目录
# os.chdir(r'C:/Users/yves1/Desktop/')

# SKU配置信息
# sku_data = pd.read_json(r'D:\GitHub\Jupyter\DataSet\sku_data.json')
sku_data = pd.read_json(r'/Users/yves/Documents/Github/Jupyter/DataSet/sku_data.json')

# 数据库配置信息
db_engine = str(r'mysql+mysqldb://%s:' + '%s' + '@%s/%s') % (sku_data.loc['USER', 'Data'], sku_data.loc['PWD', 'Data'], sku_data.loc['IP', 'Data'], sku_data.loc['DB', 'Data'])

# 表名信息
shark_gray_business_us = 'tbl_shark_gray_business_us'
shark_gray_ad_us = 'tbl_shark_gray_ad_us'

## 业务报表

In [4]:
def parse_business_report(date, table_name):
    business_report = pd.read_csv(next(value for value in os.listdir() if value.startswith('BusinessReport')))
    business_report.drop(['Title', 'Session Percentage', 'Page Views Percentage'], axis=1, inplace=True)
    business_report.rename({'(Child) ASIN': 'SKU'}, axis=1, inplace=True)
    business_report['SKU'] = business_report['SKU'].apply(lambda x: sku_data.loc[x.strip(), 'Data'])
    business_report.insert(loc=0, column='Date', value=pd.to_datetime(date, format='%Y/%m/%d'))
    engine = create_engine(db_engine)
    try:
        business_report.to_sql(table_name, engine, if_exists='append', index=False)
    except Exception as e:
        print(e)
    finally:
        engine.dispose()

## 广告报表

In [5]:
def parse_sponsored_report(date, table_name):
    sponsored_report = pd.read_excel(next(value for value in os.listdir() if value.startswith('Sponsored Products')))
    sponsored_report = sponsored_report[sponsored_report['Date']==date]
    sponsored_report.drop(['Currency', 'Ad Group Name', 'Total Return on Advertising Spend (RoAS)', '7 Day Advertised SKU Units (#)', '7 Day Other SKU Units (#)', '7 Day Advertised SKU Sales ', '7 Day Other SKU Sales '], axis=1, inplace=True)
    sponsored_report.rename({'Click-Thru Rate (CTR)': 'CTR', 'Cost Per Click (CPC)': 'CPC', '7 Day Total Sales ': 'Total Sales', 'Total Advertising Cost of Sales (ACoS) ': 'ACoS', '7 Day Total Orders (#)': 'Total Orders', '7 Day Total Units (#)': 'Total Units', '7 Day Conversion Rate': 'CR'}, axis=1, inplace=True)
    sponsored_report['ACoS'].fillna(value=0, inplace=True)
    sponsored_report[['CTR', 'CPC', 'ACoS', 'CR']] = sponsored_report[['CTR', 'CPC', 'ACoS', 'CR']].apply(lambda x: round(x, 4))
    sponsored_report.sort_values(by=['Portfolio name', 'Campaign Name'], inplace=True)
    engine = create_engine(db_engine)
    try:
        sponsored_report.to_sql(table_name, engine, if_exists='append', index=False)
    except Exception as e:
        print(e)
    finally:
        engine.dispose()

In [236]:
# 解析业务报表数据
# parse_business_report('2020/12/27', shark_gray_business_us)

# 解析赞助广告数据
# parse_sponsored_report('2020/12/29', shark_gray_ad_us)

# 数据统计

In [97]:
product_name = {'B08BXTW512': 'Women`s Thongs', 'B08GGBZLXQ': 'Men`s Boxer Briefs', 'B08K8PR7DK': 'women yoga leggings', 'B08M64C67V': 'men sweatshirt', 'B08M5GH26F': 'women vest', 'B08M5WMB78': 'women jacket'}

In [110]:
date = '2020-12-1'
engine = create_engine(db_engine)

business_sql = 'select * from tbl_shark_gray_business_us where Date(Date) = "{}"'.format(date)
sponsored_sql = 'select * from tbl_shark_gray_ad_us where Date(Date) = "{}"'.format(date)

business_data = pd.read_sql_query(business_sql, engine)
sponsored_data = pd.read_sql_query(sponsored_sql, engine)
# 转换数据格式
business_data.iloc[:, [5, 8, 9]] = business_data.iloc[:, [5, 8, 9]].apply(lambda x: x.str.replace('%', '')).astype('float')/100
business_data.iloc[:, [10, 11]] = business_data.iloc[:, [10, 11]].apply(lambda x: x.str.replace('$', '')).astype('float')

business_data.iloc[:, 1] = business_data.iloc[:, 1].apply(lambda x: product_name[x])
business_data.rename({'(Parent) ASIN': 'Product'}, axis=1, inplace=True)

business_data.drop(['SKU', 'Buy Box Percentage'], axis=1, inplace=True)

In [116]:
business_total = business_data.groupby('Product')[['Sessions', 'Page Views', 'Units Ordered', 'Units Ordered - B2B', 'Ordered Product Sales', 'Ordered Product Sales - B2B', 'Total Order Items', 'Total Order Items - B2B']].sum()
sponsored_total = sponsored_data.groupby('Portfolio name')[['Clicks', 'Spend', 'Total Sales']].sum()


In [118]:
sponsored_total

,Clicks,Spend,Total Sales
Portfolio name,,,
Men`s Boxer Briefs,9,10.16,15.99
Women`s Thongs,7,6.39,15.99


In [119]:
business_total

,Sessions,Page Views,Units Ordered,Units Ordered - B2B,Ordered Product Sales,Ordered Product Sales - B2B,Total Order Items,Total Order Items - B2B
Product,,,,,,,,
Men`s Boxer Briefs,21,26,2,0,34.72,0.0,2,0
Women`s Thongs,2,2,1,0,17.07,0.0,1,0


In [131]:
total = pd.concat([business_total, sponsored_total], axis = 1, keys=['business_total', 'sponsored_total'])

In [123]:
total.columns

MultiIndex([( 'business_total',                    'Sessions'),
            ( 'business_total',                  'Page Views'),
            ( 'business_total',               'Units Ordered'),
            ( 'business_total',         'Units Ordered - B2B'),
            ( 'business_total',       'Ordered Product Sales'),
            ( 'business_total', 'Ordered Product Sales - B2B'),
            ( 'business_total',           'Total Order Items'),
            ( 'business_total',     'Total Order Items - B2B'),
            ('sponsored_total',                      'Clicks'),
            ('sponsored_total',                       'Spend'),
            ('sponsored_total',                 'Total Sales')],
           )

In [129]:
type(total)

pandas.core.frame.DataFrame

In [ ]:
https://blog.csdn.net/flyfoxs/article/details/81346885

# 数据可视化

In [ ]:
sql = 'select * from tbl_shark_gray_business_us where Date(Date) between "{}" and "{}"'.format(start_date, end_date)